In [2]:
# import SimpleXMLRPCServer
from xmlrpc.server import SimpleXMLRPCServer

# import SimpleXMLRPCRequestHandler
from xmlrpc.server import SimpleXMLRPCRequestHandler


import threading

# Batasi hanya pada path /RPC2 saja supaya tidak bisa mengakses path lainnya
class RequestHandler(SimpleXMLRPCRequestHandler):
    rpc_paths = ('/RPC2',)

# Buat server
with SimpleXMLRPCServer(("192.168.0.110", 8008),
                        requestHandler=RequestHandler) as server:
    server.register_introspection_functions()

    # buat data struktur dictionary untuk menampung nama_kandidat dan hasil voting
    data = {"Ridwan Kamil" : 2 , "Sandiaga Uno" : 5}
    
    # kode setelah ini adalah critical section, menambahkan vote tidak boeh terjadi race condition
    # siapkan lock
    lock = threading.Lock()
    
    #  buat fungsi bernama vote_candidate()
    def vote_candidate(x):
        
        # critical section dimulai harus dilock
        lock.acquire()
        # jika kandidat ada dalam dictionary maka tambahkan  nilai votenya
        if x in data:
            data[x] += 1
            
        
        # critical section berakhir, harus diunlock
        lock.release()
        return data[x]
        
    
    # register fungsi vote_candidate() sebagai vote
    server.register_function(vote_candidate, 'vote')

    # buat fungsi bernama querry_result
    def querry_result():
        # critical section dimulai
        lock.acquire()
        
        # hitung total vote yang ada
        tot = 0
        for key,val in data.items():
            tot +=val
        
        # hitung hasil persentase masing-masing kandidat
        persentase = {}
        for key,val in data.items():
            persentase[key] = val / tot
        
        # critical section berakhir
        lock.release()
        return persentase
        
    # register querry_result sebagai querry
    server.register_function(querry_result, 'querry')


    print ("Server voting berjalan...")
    # Jalankan server
    server.serve_forever()

Server voting berjalan...


In [4]:
# 02.voting_client.py
# import xmlrpc bagian client saja
import xmlrpc.client

# buat stub (proxy) untuk client
s = xmlrpc.client.ServerProxy('http://192.168.0.110:8008')

# lakukan pemanggilan fungsi vote("nama_kandidat") yang ada di server
print(s.vote("Ridwan Kamil"))

# lakukan pemanggilan fungsi querry() untuk mengetahui hasil persentase dari masing-masing kandidat
print(s.querry())

# lakukan pemanggilan fungsi lain terserah Anda
print(s.system.listMethods())

4
{'Ridwan Kamil': 0.4444444444444444, 'Sandiaga Uno': 0.5555555555555556}
['querry', 'system.listMethods', 'system.methodHelp', 'system.methodSignature', 'vote']


In [ ]:
# import library socket karena akan digunakan request reply protocol sederhana
import socket

# definisikan IP dan port dari webserver yang akan kita gunakan. Port HTTP adalah 80
IP = '192.168.0.108'
PORT = 80

# buat socket bertipe TCP
_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

# lakukan binding 
_socket.bind((IP, PORT))

# socket mendengarkan
_socket.listen(1)

# tampilkan dengan print () "Server berjalan dan melayani HTTP pada port xx"
print('Server berjalan dan melayani HTTP pada port', PORT)

# loop forever
while True:
    # socket menerima koneksi
    conn, addr = _socket.accept()
    
    # socket menerima data
    data = conn.recv(1024)
    
    
    # print data hasil koneksi
    print(data.decode())
    print()
    
    # buat response sesuai spesifikasi HTTP untuk diberikan kepada client
    http_response = """\HTTP/1.1 200 OK

<html>
<head>
<title>Web Server Sederhana</title>
</head>
<body>

<h1>Heading 1</h1>
<p>Ini adalah contoh paragraf.</p>
<img src="https://www.surfertoday.com/images/stories/surfetiquette.jpg">

</body>
</html>
"""
    # kirim response kepada client dengan sendall() jangan lupa diencode response dengan utf-8 
    conn.sendall(http_response.encode())
    
    # tutup koneksi
    conn.close()

# Selamat! Kamu telah berhasil membuat web server sederhana. 

In [2]:
# no 3. Upload - Download file | server
# import SimpleXMLRPCServer
from xmlrpc.server import SimpleXMLRPCServer

# import SimpleXMLRPCRequestHandler
from xmlrpc.server import SimpleXMLRPCRequestHandler


import threading

# Batasi hanya pada path /RPC2 saja supaya tidak bisa mengakses path lainnya
class RequestHandler(SimpleXMLRPCRequestHandler):
    rpc_paths = ('/RPC2',)

# Buat server
with SimpleXMLRPCServer(("192.168.0.110", 8008),
                        requestHandler=RequestHandler) as server:
    server.register_introspection_functions()

    # buat data struktur dictionary untuk menampung nama_kandidat dan hasil voting
    data = {"Ridwan Kamil" : 2 , "Sandiaga Uno" : 5}
    
    # kode setelah ini adalah critical section, menambahkan vote tidak boeh terjadi race condition
    # siapkan lock
    lock = threading.Lock()
    
    #  buat fungsi bernama vote_candidate()
    def vote_candidate(x):
        
        # critical section dimulai harus dilock
        lock.acquire()
        # jika kandidat ada dalam dictionary maka tambahkan  nilai votenya
        if x in data:
            data[x] += 1
            
        
        # critical section berakhir, harus diunlock
        lock.release()
        return data[x]
        
    
    # register fungsi vote_candidate() sebagai vote
    server.register_function(vote_candidate, 'vote')

    # buat fungsi bernama querry_result
    def querry_result():
        # critical section dimulai
        lock.acquire()
        
        # hitung total vote yang ada
        tot = 0
        for key,val in data.items():
            tot +=val
        
        # hitung hasil persentase masing-masing kandidat
        persentase = {}
        for key,val in data.items():
            persentase[key] = val / tot
        
        # critical section berakhir
        lock.release()
        return persentase
        
    # register querry_result sebagai querry
    server.register_function(querry_result, 'querry')


    print ("Server voting berjalan...")
    # Jalankan server
    server.serve_forever()

In [3]:
# no 3. Download file | client
import xmlrpc.client

# buat stub (proxy) untuk client
s = xmlrpc.client.ServerProxy('http://192.168.0.105:8008')

data = s.send_file()
with open('hasil_download.txt', 'w') as handle:
    handle.write(data)

In [ ]:
# server.py
from xmlrpc.server import SimpleXMLRPCRequestHandler, SimpleXMLRPCServer
import os

# Batasi hanya pada path /RPC2 saja supaya tidak bisa mengakses path lainnya
class RequestHandler(SimpleXMLRPCRequestHandler):
    rpc_paths = ('/RPC2',)

# Buat server
with SimpleXMLRPCServer(("192.168.0.105", 8008),
                        requestHandler=RequestHandler) as server:
    server.register_introspection_functions()
    

    def send_file():
        with open('file_diunduh.txt', 'r') as handle:
            return handle.read()    
    
    print("File berhasil dikirim")
    server.register_function(send_file, 'send_file')
    server.serve_forever()